In [1]:
import datetime
import collections
import math
import os

from IPython.display import display
import pandas as pd
import numpy as np

# References
- [Tensorflow Mechanics 101](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/tf/index.html)
- [Running Graphs](https://www.tensorflow.org/versions/r0.9/api_docs/python/client.html)
- [Building Graphs](https://www.tensorflow.org/versions/r0.9/api_docs/python/framework.html)


In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'0.9.0'

In [4]:
x = tf.constant(3.0)
y = tf.constant(4.0)
z = x + y

print 'z ==', z
print 'z.graph ==', z.graph
print 'x.graph == z.graph: ', z.graph == y.graph
print 'x.graph == default_graph(): ', x.graph == tf.get_default_graph()
sess = tf.Session()
# 7.0
print 'run(z) ==', sess.run(z)
# Evaluate z multiple times (7.0).
print 'run(z) ==', sess.run(z)
sess.close()

# RuntimeError: Attempted to use a closed Session.
# print 'run(z) ==', sess.run(z)

z == Tensor("add:0", shape=(), dtype=float32)
z.graph == <tensorflow.python.framework.ops.Graph object at 0x10f32e4d0>
x.graph == z.graph:  True
x.graph == default_graph():  True
run(z) == 7.0
run(z) == 7.0


In [5]:
# Call close() automatically by with-statement.
# Also, with-statement calls as_default() implicitly.
with tf.Session() as sess:
  print 'run(z) ==', sess.run(z)
  print 'z.eval() ==', z.eval()

run(z) == 7.0
z.eval() == 7.0


In [6]:
sess = tf.Session()

# This is error.
# z.eval()
print 'get_default_session', tf.get_default_session()

with sess.as_default():
    print 'get_default_session', tf.get_default_session(), tf.get_default_session() == sess
    print 'z.eval() ==', z.eval()

get_default_session None
get_default_session <tensorflow.python.client.session.Session object at 0x10f323710> True
z.eval() == 7.0


# Variable and assign
- [tf.Variable](https://www.tensorflow.org/versions/r0.9/api_docs/python/state_ops.html#Variable)
- [tf.Graph.control_dependencies](https://www.tensorflow.org/versions/r0.9/api_docs/python/framework.html#Graph.control_dependencies)

In [7]:
# Why is not the spec of assign documented well in Tensorflow?
v = tf.Variable(5.0)
w = v + 1
assign = v.assign(w)

# It seems like the order of assign and other operations is not defined
# unless control_dependencies is used.
# TODO: Is there a document about this rule?
a = tf.identity(v)
b = v + 0

# control_dependencies
with tf.get_default_graph().control_dependencies([assign]):
    c = tf.identity(v)
    d = v + 0

print assign

with tf.Session() as sess:
    tf.initialize_all_variables().run()
    print '=== init ==='
    print 'v =', v.eval()
    print '=== assign ==='
    print 'assign =', assign.eval()
    print 'v =', v.eval()
    print '=== run two "assign"s'
    print sess.run([assign, assign])
    print v.eval()
    print '=== assign and v ==='
    print sess.run([v, assign])
    print sess.run([assign, v])
    print '=== a, b, c d and assign ==='
    print sess.run([a, b, c, d, assign])

Tensor("Assign:0", shape=(), dtype=float32_ref)
=== init ===
v = 5.0
=== assign ===
assign = 6.0
v = 6.0
=== run two "assign"s
[7.0, 7.0]
7.0
=== assign and v ===
[8.0, 8.0]
[9.0, 9.0]
=== a, b, c d and assign ===
[10.0, 9.0, 10.0, 10.0, 10.0]


# Placeholder

In [8]:
p = tf.placeholder(tf.float32)
q = p + 4.0

p_matrix = tf.placeholder(tf.float32, [2, 3])
# Note: This is not matmul :)
pp = p_matrix * p_matrix
mul = tf.matmul(p_matrix, tf.transpose(p_matrix))

q_matrix = tf.placeholder(tf.float32, [None, 3])
q_mul = tf.matmul(q_matrix, [[1.0], [2.0], [3.0]])

with tf.Session() as sess:
    print '=== q (p: 3.0) ==='
    print sess.run(q, {p: 3.0})
    print '=== q (p: 10.0) ==='
    print q.eval({p: 10.0})
    
    print '=== matrix ==='
    print sess.run(pp, {p_matrix: [[1,2,3], [4,5,6]]})
    print sess.run(mul, {p_matrix: [[1,2,3], [4,5,6]]})
    
    print '=== None in shape of placeholder ==='
    print sess.run(q_mul, {q_matrix: [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]})

=== q (p: 3.0) ===
7.0
=== q (p: 10.0) ===
14.0
=== matrix ===
[[  1.   4.   9.]
 [ 16.  25.  36.]]
[[ 14.  32.]
 [ 32.  77.]]
=== None in shape of placeholder ===
[[ 14.]
 [ 32.]]


# Optimization

In [9]:
x = tf.Variable(3.5)
y = x - tf.constant(4.0)
step = tf.train.GradientDescentOptimizer(0.1).minimize(tf.abs(y))
print step
print isinstance(step, tf.Operation)

print '===== default session ====='
with tf.Session():
    # FailedPreconditionError: Attempting to use uninitialized value 
    # x.eval()
    init = tf.initialize_all_variables()
    init.run()
    for i in xrange(10):
        step.run()
        print x.eval()

print '===== explicit session ====='
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)
for i in xrange(10):
    sess.run(step)
    print sess.run(x)  # sess does not have 'eval'. Use run.
sess.close()

print '===== explicit session as arg====='
sess = tf.Session()
init = tf.initialize_all_variables()
init.run(session=sess)
for i in xrange(10):
    step.run(session=sess)
    print x.eval(session=sess)
sess.close()

name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_Variable_1/ApplyGradientDescent"

True
===== default session =====
3.6
3.7
3.8
3.9
4.0
4.1
4.0
4.1
4.0
4.1
===== explicit session =====
3.6
3.7
3.8
3.9
4.0
4.1
4.0
4.1
4.0
4.1
===== explicit session as arg=====
3.6
3.7
3.8
3.9
4.0
4.1
4.0
4.1
4.0
4.1


# Graph
- A session is under a graph
- Methods to generate tensors (like constant) do not have 'graph' argument unlike eval/run and session.

In [10]:
g0 = tf.Graph()
g1 = tf.Graph()
with g0.as_default():
    c0 = tf.constant(3.0)
    c1 = tf.constant(4.0)
    sess0 = tf.Session()
with g1.as_default():
    c2 = tf.constant(4.0)
    sess1 = tf.Session()

print 'c0, c1, c2 =', c0, c1, c2
print 'Graph equality:', c0.graph == c1.graph, c0.graph == c2.graph

print 'c0.eval ==', c0.eval(session=sess0)

# Error!
# ValueError: Cannot use the given session to evaluate tensor: the tensor's graph is different from the session's graph.
# print 'c0.eval ==', c0.eval(session=sess1)

# 'c0 + c1' works even if g0 is not 'default'.
print '==== (c0 + c1) ===='
print (c0 + c1).eval(session=sess0)
print (c0 + c1).graph == c0.graph

# Error!
# ValueError: Tensor("Const:0", shape=(), dtype=float32) must be from the same graph as Tensor("Const:0", shape=(), dtype=float32).
# c2 = c0 + c1

sess0.close()
sess1.close()
print '=== finalize Graphs ==='
# Graphs are not finalized.
print g0.finalized, g1.finalized
g0.finalize()
g1.finalize()
print g0.finalized, g1.finalized

with g0.as_default():
    # RuntimeError: Graph is finalized and cannot be modified.
    # tf.constant(5.0)
    pass

c0, c1, c2 = Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32) Tensor("Const:0", shape=(), dtype=float32)
Graph equality: True False
c0.eval == 3.0
==== (c0 + c1) ====
7.0
True
=== finalize Graphs ===
False False
True True


In [11]:
# https://www.tensorflow.org/versions/r0.9/api_docs/python/framework.html#name_scope
# https://www.tensorflow.org/versions/r0.9/api_docs/python/framework.html#Graph.name_scope
with tf.name_scope('my_scope'):
    c = tf.constant(1.0, name='my_const')

print c

Tensor("my_scope/my_const:0", shape=(), dtype=float32)


# What are Tensor, Variable and Operation?
- [tf.Tensor](https://www.tensorflow.org/versions/r0.9/api_docs/python/framework.html#Tensor)
- [tf.Variable](https://www.tensorflow.org/versions/r0.9/api_docs/python/framework.html#Variable)
- [tf.Operation](https://www.tensorflow.org/versions/r0.9/api_docs/python/framework.html#Operation)


In [12]:
tensor = pd.Series()
operation = pd.Series()
variable = pd.Series()

def check_type(label, v):
    tensor[label] = isinstance(v, tf.Tensor)
    operation[label] = isinstance(v, tf.Operation)
    variable[label] = isinstance(v, tf.Variable)

x = tf.constant(3.0)
y = tf.constant(4.0)
z = x + y
v = tf.Variable(1.0)
w = v + x
# This is Tensor
assign = v.assign(w)

# This is Operation.
step = tf.train.GradientDescentOptimizer(0.1).minimize(w)

# This is Operation too.
init = tf.initialize_all_variables()

p = tf.placeholder(tf.float64)

mat = tf.constant([[1, 2], [3, 4]])
mul = tf.matmul(mat, mat)

check_type('x', x)
check_type('z', z)
check_type('init', init)
check_type('p', p)
check_type('v', v)
check_type('w', w)
check_type('assign', assign)
check_type('step', step)

check_type('mat', mat)
# It seems like the comment in
# https://www.tensorflow.org/versions/r0.9/api_docs/python/framework.html#Operation
# is wrong.
# "Objects of type Operation are created by calling a Python op constructor (such as tf.matmul())"
check_type('mul', mul)

df = pd.DataFrame(collections.OrderedDict([('Tensor', tensor), ('Variable', variable), ('Operation', operation)]))
df.applymap(lambda b:u'\u2713' if b else u'')

,Tensor,Variable,Operation
x,✓,,
z,✓,,
init,,,✓
p,✓,,
v,,✓,
w,✓,,
assign,✓,,
step,,,✓
mat,✓,,
mul,✓,,


# Random and Session

In [13]:
mat0 = tf.random_normal([2,2])
mat1 = mat0 + 1
with tf.Session() as sess:
    # mat1.eval() != mat0.eval() + 1
    display(mat0.eval())
    display(mat1.eval())
    # mat1.eval() == mat0.eval() + 1
    display(sess.run((mat0, mat1)))

print '----- Random variable -------'
mat2 = tf.Variable(tf.random_normal([2, 2]))
with tf.Session():
    # Do not forget to initialize variables.
    tf.initialize_all_variables().run()
    # mat2.eval() == mat2.eval().
    display(mat2.eval())
    display(mat2.eval())

array([[ 1.0328182 ,  0.34343874],
       [-0.47436199, -2.49098158]], dtype=float32)

array([[ 0.42207646,  1.74090791],
       [ 1.78141737,  1.68370342]], dtype=float32)

[array([[-1.14442492, -0.75665456],
        [-0.54312569,  2.40435791]], dtype=float32),
 array([[-0.14442492,  0.24334544],
        [ 0.45687431,  3.40435791]], dtype=float32)]

----- Random variable -------


array([[ 0.67183959, -1.25063252],
       [-0.66712266,  0.7336309 ]], dtype=float32)

array([[ 0.67183959, -1.25063252],
       [-0.66712266,  0.7336309 ]], dtype=float32)

# Saver
Saver saves and restores variables.
- [Saver](https://www.tensorflow.org/versions/r0.9/api_docs/python/state_ops.html#Saver)

In [14]:
try:
    os.mkdir('tf_saver_dir')
except OSError:
    pass

v = tf.Variable(0.0)
assign = v.assign_add(1.0)

saver = tf.train.Saver()
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    for step in xrange(10):
        assign.eval()
        print 'saved(step=%d):' % step, saver.save(sess, 'tf_saver_dir/saved', global_step=step)
    
    tf.initialize_all_variables().run()
    print '=== v after reset ==='
    print v.eval()

    print '=== Show not-yet-deleted checkpoints ==='
    # Old checkpoints are deleted automatically.
    # The default value of max_to_keep is 5.
    print saver.last_checkpoints

    print '=== v after restore ==='
    # Pass 'step' as a part of prefix string.
    saver.restore(sess, 'tf_saver_dir/saved-7')
    print v.eval()

saved(step=0): tf_saver_dir/saved-0
saved(step=1): tf_saver_dir/saved-1
saved(step=2): tf_saver_dir/saved-2
saved(step=3): tf_saver_dir/saved-3
saved(step=4): tf_saver_dir/saved-4
saved(step=5): tf_saver_dir/saved-5
saved(step=6): tf_saver_dir/saved-6
saved(step=7): tf_saver_dir/saved-7
saved(step=8): tf_saver_dir/saved-8
saved(step=9): tf_saver_dir/saved-9
=== v after reset ===
0.0
=== Show not-yet-deleted checkpoints ===
['tf_saver_dir/saved-5', 'tf_saver_dir/saved-6', 'tf_saver_dir/saved-7', 'tf_saver_dir/saved-8', 'tf_saver_dir/saved-9']
=== v after restore ===
8.0


# GraphDef
How to serialize/deserialize Tensorflow Graph and variables. Suprisingly, this is not well documented in Tensorflow manual as of July 2016.

## Notes
- graph.as_graph_def returns GraphDef [protocol buffer](https://developers.google.com/protocol-buffers/docs/pythontutorial)
  - GraphDef is defined in `tensorflow.core.framework.graph_pb2`
  - *GraphDef* does not include the contents of variables. You need to store the contents of variables by Saver separately.
- TODO:
  - Understand why we need to use `x:0` rather than `x` ([Stackoverflow](http://stackoverflow.com/a/37147387)).
  - `restore_op` part is tricky. I copied this code from freeze_graph.py. Find the document to help understand this part.

## References
- [A Tool Developer's Guide to TensorFlow Model Files](https://www.tensorflow.org/versions/r0.9/how_tos/tool_developers/index.html)
- [tf.import_graph_def](https://www.tensorflow.org/versions/r0.9/api_docs/python/framework.html#import_graph_def)
- [Code in freeze_graph.py](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/tools/freeze_graph.py)

In [15]:
from tensorflow.core.framework.graph_pb2 import GraphDef

graph = tf.Graph()
with graph.as_default():
    x = tf.Variable(3.0, dtype=tf.float32, name='x')
    y = tf.placeholder(tf.float32, name='y')
    z = tf.add(x, y, name='z')
    
    init = tf.initialize_all_variables()
    saver = tf.train.Saver(name='mysaver')

with tf.Session(graph=graph) as sess:
    init.run()
    saver.save(sess, 'tf_saver_dir/graph-vars')

# Convert to GraphDef protobuf.
graph_proto = graph.as_graph_def()
binary = graph_proto.SerializeToString()

#### Deserialize ####

deserialized = GraphDef()
deserialized.MergeFromString(binary)

new_graph = tf.Graph()
# Import serialized graph to new_graph.
with new_graph.as_default():
    x, y, z, restore_op, filename = tf.import_graph_def(
        deserialized, return_elements=[
            'x:0', 'y:0', 'z:0', 'mysaver/restore_all', 'mysaver/Const:0'])

print 'x.graph == new_graph:', x.graph == new_graph

with tf.Session(graph=new_graph) as sess:
    # Error! "Attempting to use uninitialized value".
    # print x.eval()
    sess.run([restore_op], {filename: 'tf_saver_dir/graph-vars'})
    print 'z ==', sess.run(z, {y: 4.0})

x.graph == new_graph: True
z == 7.0


# TensorBoard
Run the following code block then open tensorboard with

    tensorboard --logdir /your/dir/tensorflow_summary --reload_interval 5

## Tips
- [TensorBoard Tutorial](https://www.tensorflow.org/versions/r0.9/how_tos/summaries_and_tensorboard/index.html)
- [tf.SummaryWriter](https://www.tensorflow.org/versions/r0.9/api_docs/python/train.html#SummaryWriter)
  - Pass flush_secs arg to the constructor to control flush intervals.
  - You need to change the output directory every run. Otherwise, summaries from multiple runs are mixed and sometimes are overwritten and gone.
  - You should call `close` to flush data immediately.
    - It helped me notice the problem above.
- [datetime.strftime](# https://docs.python.org/2/library/datetime.html#strftime-strptime-behavior
)

In [16]:
BATCH_SIZE = 100

dataset = collections.namedtuple('dataset', ('inputs', 'outputs'))

def create_dataset(size):
    inputs = np.random.uniform(0.0, 10.0, [size, 3])
    outputs = np.matmul(inputs, np.array([3, 4, 2])) - 5
    outputs += np.random.normal(scale=0.5, size=[size])
    return dataset(inputs, outputs)

data = collections.namedtuple('data', ('train', 'test'))(
    create_dataset(1000 * BATCH_SIZE),
    create_dataset(100 * BATCH_SIZE)
)

def train_batch_generator():
    index = 0
    while True:
        yield (data.train.inputs[index:index + BATCH_SIZE],
               data.train.outputs[index:index + BATCH_SIZE])
        index += BATCH_SIZE
        if index >= data.train.inputs.shape[0]:
            index = 0
train_batch = train_batch_generator()

def now_str():
    return datetime.datetime.now().strftime('%Y%m%d_%H%M%S_%f')

graph = tf.Graph()
sess = tf.Session(graph=graph)
with graph.as_default():
    inputs = tf.placeholder(tf.float32, [None, 3])
    outputs = tf.placeholder(tf.float32, [None])

    weights = tf.Variable(tf.truncated_normal([3, 1], stddev=1))
    bias = tf.Variable(0.0)
    pred = tf.matmul(inputs, weights) + bias
    loss = tf.reduce_mean(tf.square(pred - outputs))
    # Do not forget to reshape!
    # Again, do not forget to reshape!
    diff = pred - tf.reshape(outputs, shape=[-1, 1])
    loss = tf.reduce_mean(tf.square(diff))
    
    optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    
    # Summary
    tf.scalar_summary('loss', loss)
    for i in xrange(3):
        tf.scalar_summary('coef/%d' % i, weights[i, 0])
    tf.scalar_summary('bias', bias)
    merged = tf.merge_all_summaries()
    
    # You do not need to pass `graph` if you do not need to show "Graph" in TensorBoard.
    writer = tf.train.SummaryWriter(
        'tensorflow_summary/test/%s' % now_str(),
        flush_secs=10,
        graph=graph)
    
    init_variables = tf.initialize_all_variables()


with sess.as_default():
    init_variables.run()
    
    for step in xrange(2500):
        batch_inputs, batch_outputs = train_batch.next()
        optimizer.run({inputs: batch_inputs, outputs: batch_outputs})
        if step % 10 == 0 and step > 0:
            summary = merged.eval({inputs: data.test.inputs, outputs: data.test.outputs})
            writer.add_summary(summary, global_step=step)
    print 'Weight:', weights.eval()
    print 'Bias:', bias.eval()

# close to flush data immediately.
writer.close()

Weight: [[ 3.009408  ]
 [ 4.01078558]
 [ 2.01212025]]
Bias: -4.94595
